In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
from tqdm.notebook import tqdm, trange

"""Change to the data folder"""
new_path = "./new_train/new_train"

cuda_status = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 64

def my_collate_x(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    #inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    inp = [numpy.dstack([scene['p_in'][:,:,0]]) for scene in batch]
    #out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][:,:,0]]) for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    return [inp, out]

def my_collate_y(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    #inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    inp = [numpy.dstack([scene['p_in'][:,:,1]]) for scene in batch]
    #out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][:,:,1]]) for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    return [inp, out]

def test_collate_x(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][:,:,0]]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, scene_ids, track_ids, agent_ids]

def test_collate_y(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][:,:,1]]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, scene_ids, track_ids, agent_ids]

train_loader_x = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_x, num_workers=0)
train_loader_y = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_y, num_workers=0)

In [4]:
class LSTMEncoderDecoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(60, hidden_size=100, batch_first=True)
        self.lstm2 = torch.nn.LSTM(100, hidden_size=100, batch_first=True, num_layers=2)
        self.linear = torch.nn.Linear(1900, 1800)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.relu(x)
        x, _ = self.lstm2(x)
        x = self.relu(x)
        x = torch.flatten(x, start_dim=1)
        x = self.relu(self.linear(x))
        x = torch.reshape(x, torch.Size([64, 60, 30]))
        return x

x_model = LSTMEncoderDecoder()
x_model.to(device)
if cuda_status:
    x_model = x_model.cuda()
y_model = LSTMEncoderDecoder()
y_model.to(device)
if cuda_status:
    y_model = y_model.cuda()

In [ ]:
model = torch.load('./models/1epochAdam.pt')
model.train()
model.to(device)
if cuda_status:
    model = model.cuda()

### Visualize the batch of sequences

In [ ]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm

agent_id = 0
epoch = 3
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
#learning_rate =0.01
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
x_optimizer = torch.optim.Adam(x_model.parameters(), lr=learning_rate)
y_optimizer = torch.optim.Adam(y_model.parameters(), lr=learning_rate)
iterator_x = train_loader_x
iterator_y = train_loader_y

for i in trange(epoch):
    
    for (i_batch, sample_batch_x), (j_batch, sample_batch_y) in tqdm(zip(enumerate(iterator_x), enumerate(iterator_y)), total=3216):
        inp_x, out_x = sample_batch_x
        inp_y, out_y = sample_batch_y

        in_x = torch.flatten(inp_x, start_dim=2)
        out_x = torch.flatten(out_x, start_dim=2)
        in_y = torch.flatten(inp_y, start_dim=2)
        out_y = torch.flatten(out_y, start_dim=2)
        
        in_x = in_x.permute(0,2,1)
        #out_x = out_x.permute(0,2,1)
        in_y = in_y.permute(0,2,1)
        #out_y = out_y.permute(0,2,1)
        
        in_x = in_x.float()
        out_x = out_x.float()
        in_y = in_y.float()
        out_y = out_y.float()

        if cuda_status:
            in_x.to(device)
            out_x.to(device)
            in_y.to(device)
            out_y.to(device)
            
            in_x = in_x.cuda()
            out_x = out_x.cuda()
            in_y = in_y.cuda()
            out_y = out_y.cuda()

        x_pred = None
        # Forward pass: predict y by passing x to the model.    
        x_pred = x_model(in_x)
        #y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        x_loss = loss_fn(x_pred, out_x)

        # Before backward pass, zero outgradients to clear buffers  
        x_optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        x_loss.backward()

        # makes an gradient descent step to update its parameters
        x_optimizer.step()
        
        y_pred = None
        # Forward pass: predict y by passing x to the model.    
        y_pred = y_model(in_y)
        #y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        y_loss = loss_fn(y_pred, out_y)

        # Before backward pass, zero outgradients to clear buffers  
        y_optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        y_loss.backward()

        # makes an gradient descent step to update its parameters
        y_optimizer.step()
        
        print(str(torch.sqrt(x_loss)) + " " + str(torch.sqrt(y_loss)))

        if i_batch == 3216:
            break


tensor(799.2502, device='cuda:0', grad_fn=<SqrtBackward>) tensor(879.0607, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(781.5557, device='cuda:0', grad_fn=<SqrtBackward>) tensor(877.2045, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(965.4304, device='cuda:0', grad_fn=<SqrtBackward>) tensor(796.4229, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(721.6404, device='cuda:0', grad_fn=<SqrtBackward>) tensor(948.8709, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(695.2684, device='cuda:0', grad_fn=<SqrtBackward>) tensor(926.2650, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(752.9189, device='cuda:0', grad_fn=<SqrtBackward>) tensor(942.9468, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(582.8083, device='cuda:0', grad_fn=<SqrtBackward>) tensor(876.5192, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(734.6807, device='cuda:0', grad_fn=<SqrtBackward>) tensor(863.1770, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(747.9660, device='cuda:0', grad_fn=<SqrtBackward>) tensor(1009.46

tensor(824.5688, device='cuda:0', grad_fn=<SqrtBackward>) tensor(806.2856, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(730.3732, device='cuda:0', grad_fn=<SqrtBackward>) tensor(924.0704, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(649.9883, device='cuda:0', grad_fn=<SqrtBackward>) tensor(933.8777, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(671.6512, device='cuda:0', grad_fn=<SqrtBackward>) tensor(778.2968, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(643.1281, device='cuda:0', grad_fn=<SqrtBackward>) tensor(893.8199, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(675.8730, device='cuda:0', grad_fn=<SqrtBackward>) tensor(861.2225, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(853.3851, device='cuda:0', grad_fn=<SqrtBackward>) tensor(757.0832, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(645.4004, device='cuda:0', grad_fn=<SqrtBackward>) tensor(914.8071, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(739.9238, device='cuda:0', grad_fn=<SqrtBackward>) tensor(877.839

tensor(695.2014, device='cuda:0', grad_fn=<SqrtBackward>) tensor(803.7007, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(764.1124, device='cuda:0', grad_fn=<SqrtBackward>) tensor(761.4576, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(805.8029, device='cuda:0', grad_fn=<SqrtBackward>) tensor(817.6094, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(744.6180, device='cuda:0', grad_fn=<SqrtBackward>) tensor(786.5547, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(723.5813, device='cuda:0', grad_fn=<SqrtBackward>) tensor(821.0203, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(762.3081, device='cuda:0', grad_fn=<SqrtBackward>) tensor(856.5100, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(655.3928, device='cuda:0', grad_fn=<SqrtBackward>) tensor(972.5924, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(710.3427, device='cuda:0', grad_fn=<SqrtBackward>) tensor(702.3506, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(576.1914, device='cuda:0', grad_fn=<SqrtBackward>) tensor(968.045

tensor(611.8132, device='cuda:0', grad_fn=<SqrtBackward>) tensor(855.0745, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(758.6752, device='cuda:0', grad_fn=<SqrtBackward>) tensor(783.0179, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(651.7525, device='cuda:0', grad_fn=<SqrtBackward>) tensor(884.6627, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(613.9224, device='cuda:0', grad_fn=<SqrtBackward>) tensor(898.3222, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(757.9352, device='cuda:0', grad_fn=<SqrtBackward>) tensor(916.4980, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(737.6208, device='cuda:0', grad_fn=<SqrtBackward>) tensor(848.6558, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(716.0724, device='cuda:0', grad_fn=<SqrtBackward>) tensor(887.8046, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(655.4786, device='cuda:0', grad_fn=<SqrtBackward>) tensor(823.0251, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(714.7269, device='cuda:0', grad_fn=<SqrtBackward>) tensor(670.690

tensor(774.8439, device='cuda:0', grad_fn=<SqrtBackward>) tensor(866.2943, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(624.9739, device='cuda:0', grad_fn=<SqrtBackward>) tensor(817.7437, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(711.4099, device='cuda:0', grad_fn=<SqrtBackward>) tensor(760.1830, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(750.2084, device='cuda:0', grad_fn=<SqrtBackward>) tensor(760.3333, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(659.0033, device='cuda:0', grad_fn=<SqrtBackward>) tensor(833.3519, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(754.4535, device='cuda:0', grad_fn=<SqrtBackward>) tensor(859.2501, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(687.1833, device='cuda:0', grad_fn=<SqrtBackward>) tensor(812.0627, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(698.1463, device='cuda:0', grad_fn=<SqrtBackward>) tensor(922.0418, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(679.0558, device='cuda:0', grad_fn=<SqrtBackward>) tensor(886.381

tensor(715.0811, device='cuda:0', grad_fn=<SqrtBackward>) tensor(772.9039, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(589.0088, device='cuda:0', grad_fn=<SqrtBackward>) tensor(879.8939, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(628.4825, device='cuda:0', grad_fn=<SqrtBackward>) tensor(902.6805, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(719.1678, device='cuda:0', grad_fn=<SqrtBackward>) tensor(814.6317, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(569.0098, device='cuda:0', grad_fn=<SqrtBackward>) tensor(972.0096, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(576.4651, device='cuda:0', grad_fn=<SqrtBackward>) tensor(796.0955, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(680.3384, device='cuda:0', grad_fn=<SqrtBackward>) tensor(709.4492, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(763.5768, device='cuda:0', grad_fn=<SqrtBackward>) tensor(783.2244, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(677.1470, device='cuda:0', grad_fn=<SqrtBackward>) tensor(838.865

In [ ]:
torch.save(model, './models/2epochCNN-LSTM.pt')

In [ ]:
model = torch.load('./models/6epochmodel.pt')
model.eval()
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
import pandas as pd

# Submission output
writeCSV = True
val_path = "./new_val_in/new_val_in"

if writeCSV:
    
    dataset = ArgoverseDataset(data_path=val_path)
    test_loader = DataLoader(dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)
    
    data = []
    
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(tqdm(test_loader)):
            inp, scene_ids, track_ids, agent_ids = sample_batch

            if cuda_status:
                model = model.cuda()
                x = inp.permute(0,3,1,2).float().cuda()
            else:
                x = inp.permute(0,3,1,2).float()

            y_pred = None

            # Forward pass: predict y by passing x to the model.    
            y_pred = model(x)
            y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))
            
            for i in range(batch_sz):
                row = []
                row.append(scene_ids[i].item())
                curr = y_pred[i]
                
                agent_id = agent_ids[i]
                
                for j in range(30):
                    vehicle_index = 0
                    found = False
                    while not found:
                        if track_ids[i][vehicle_index][j][0] == agent_id:
                            found = True
                        else:
                            vehicle_index += 1

                    row.append(str(curr[vehicle_index][j][0].item()))
                    row.append(str(curr[vehicle_index][j][1].item()))
                    
                data.append(row)

    df = pd.DataFrame(data, columns = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35','v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52','v53','v54','v55','v56','v57','v58','v59','v60'])
    print(df)
    df.to_csv('submission.csv', index=False)
                
                
                